<a href="https://colab.research.google.com/github/Kobayashi139/food-supply/blob/main/%E3%83%AD%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0%E3%83%9B%E3%83%A9%E3%82%A4%E3%82%BA%E3%83%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ライブラリ

In [11]:
# =================================================================
# モジュール1: 環境設定とライブラリのインポート
# =================================================================
!pip install -q pyomo plotly
!apt-get install -y -qq glpk-utils

import math
import pyomo.environ as pyo # 数理最適化
import pandas as pd # テーブル操作
import numpy as np # 数値計算、配列など
import random, time # 乱数、速度確認
import plotly.express as px # グラフ描画（簡単
import plotly.graph_objects as go # グラフ描画（本格
from plotly.subplots import make_subplots # グラフを並べる

In [12]:
try:
    # 乱数シードを固定し、毎回同じ乱数が発生するようにする
    np.random.seed(42)
    solver = pyo.SolverFactory('glpk')
except pyo.common.errors.ApplicationError: # error時
    print("GLPK solver not found. Please ensure it is installed and in your system's PATH.")
    solver = None  # null

## パラメータ

In [13]:
# =================================================================
# モジュール2: パラメータとシナリオの定義
# =================================================================

# 小数点第2位で切り上げるための補助関数
def ceil_decimal(value, n=2):
    return math.ceil(value * (10**n)) / (10**n)

# --- エネルギー ---
ELECTRICITY_RATE = 21  # 電気料金単価 (円/kWh)
LP_GAS_RATE = 366      # LPガス単価 (円/kg)

# --- 製品タイプの集合 ---
products = ['f', 'a', 'b']
processed_products = ['a', 'b']

# --- 単位統一のための基本データ設定 ---
# 檸檬('f')
WEIGHT_LEMON_kg = 0.1       # 1個当たり100g
LIFESPAN_FRESH_days = 60    # 保存期間 (日)
SALES_FRESH = 500           # 値段 (円/kg)

# タルト('a')
WEIGHT_TART_g = 350         # 容量 (g/個)
LEMON_PER_TART_ml = 50      # 檸檬含量 (ml/個)
LIFESPAN_TART_days = 100    # 保存期間 (日)
SALES_TART = 3800           # 値段 (円/個)

# シロップ/ジャム('b')
WEIGHT_SYRUP_g = 400        # 容量 (g/個)
LEMON_PER_SYRUP_ml = 200    # 檸檬含量 (ml/個)
LIFESPAN_SYRUP_days = 365   # 保存期間 (日)
SALES_SYRUP = 2200          # 値段 (円/個)

# 製品1個の総重量 (kg)
WEIGHT_PIECE = {
    'a': WEIGHT_TART_g / 1000,
    'b': WEIGHT_SYRUP_g / 1000
}

# 製品1個に含まれるレモン重量 (kg-lemon)
LEMON_PER_PIECE = {
    'a': LEMON_PER_TART_ml / 1000,
    'b': LEMON_PER_SYRUP_ml / 1000
}

# 販売価格の変換 (円 / kg-lemon)
sales_prices = {
    'f': SALES_FRESH ,
    'a': SALES_TART / LEMON_PER_PIECE['a'] , # 3800円 / 0.05kg = 76,000円
    'b': SALES_SYRUP / LEMON_PER_PIECE['b']
}

# --- 設備スペックに基づくコスト算出 ---

# 保管コスト h (円/kg・月)
# 冷蔵庫: 14,000円/年, 1049L
COST_REF_monthly = 14000 / 12
CAPACITY_REF_kg = 1049 * 0.4  # 容積の40%程度が実有効重量
h_f = ceil_decimal(COST_REF_monthly / CAPACITY_REF_kg)

# 冷凍庫: 34,400円/年, 616L
COST_FREEZER_monthly = 34400 / 12
CAPACITY_FREEZER_kg = 616 * 0.4
h_a_phys = ceil_decimal(COST_FREEZER_monthly / CAPACITY_FREEZER_kg)
h_a = ceil_decimal(h_a_phys * (WEIGHT_PIECE['a'] / LEMON_PER_PIECE['a']))

# 常温(シロップ)
h_b_phys = 1 # 管理費として微小値を設定
h_b = ceil_decimal(h_b_phys * (WEIGHT_PIECE['b'] / LEMON_PER_PIECE['b']))

# 加工コスト p (円/kg-input)
# タルト: スチームコンベクション 5.9kW 0.5時間、梱包63円
cost_energy_a = 5.9 * 0.5 * ELECTRICITY_RATE + (63 * 30) # 30個分
p_a = ceil_decimal(cost_energy_a / (30 * LEMON_PER_PIECE['a']))

# ジャム: ２口ガステーブル 0.915kg/h, 1時間、梱包230円
cost_energy_b = 0.915 * 1 * LP_GAS_RATE + (230 * 25) # 25個分
p_b = ceil_decimal(cost_energy_b / (25 * LEMON_PER_PIECE['b']))

# 劣化率 gamma (%/月)
gamma_f = ceil_decimal(min(1.0, 30 / LIFESPAN_FRESH_days))
gamma_a = ceil_decimal(min(1.0, 30 / LIFESPAN_TART_days))
gamma_b = ceil_decimal(min(1.0, 30 / LIFESPAN_SYRUP_days))

# --- 廃棄単価の変換 (円 / kg-lemon) ---
# レモン1kgを捨てる時、実際には製品全体の重さ（他材料含む）を捨てるコストがかかる
phi_f = 22.8  # 生鮮レモンはそのまま (1.0倍)
phi_a = phi_f * (WEIGHT_PIECE['a'] / LEMON_PER_PIECE['a']) # 7.0倍
phi_b = phi_f * (WEIGHT_PIECE['b'] / LEMON_PER_PIECE['b']) # 2.0倍

# h: 保管コスト, p: 加工コスト, gamma: 劣化率, e: 加工効率, phi: 廃棄コスト, o:機会損失コスト
params = {
    'h': {'f': h_f, 'a': h_a, 'b': h_b},
    'p': {'f': 0, 'a': p_a, 'b': p_b},
    'gamma': {'f': gamma_f, 'a': gamma_a, 'b': gamma_b},
    'e': {'f': 1, 'a': 0.7, 'b': 0.9},
    'o': sales_prices,
    'phi': {'f': phi_f, 'a': phi_a, 'b': phi_b}
}

# --- 能力に関するパラメータ (単位: kg) ---
capacities = {
    'U': {
        'f': 400, # 冷蔵庫
        'a': CAPACITY_FREEZER_kg * (LEMON_PER_PIECE['a'] / WEIGHT_PIECE['a']), # 冷凍庫(レモン換算)
        'b': 200 * LEMON_PER_PIECE['b']  # 常温棚
    },
    'limit_units': { # 月間製造上限(個数)
        'a': 500 * LEMON_PER_PIECE['a'],
        'b': 60 * LEMON_PER_PIECE['b']
    }
}

# 確認
print(f"保管コスト: {params['h']}")
print(f"加工コスト: {params['p']}")
print(f"機会損失コスト: {params['o']}")
print(f"廃棄コスト: {params['phi']}")
print(f"劣化率: {params['gamma']}")

# --- 需要パラメータ ---
demand_params = {
    'mean': {
        'f': 5.0,
        'a': 120 * LEMON_PER_PIECE['a'],
        'b': 35 * LEMON_PER_PIECE['b']
    },
    'std': {
        'f': 2.0,
        'a': 40 * LEMON_PER_PIECE['a'],
        'b': 10 * LEMON_PER_PIECE['b']
    }
}
print(f"需要パラメータ{demand_params}")

# --- 供給と初期在庫 (単位: kg) ---
C_supply = 25 # 月間の生鮮品(檸檬)の供給量 (kg)

# 加工後在庫
initial_inventory_post = {
    'f': 6.0,
    'a':  ceil_decimal(5 * LEMON_PER_PIECE['a']),
    'b':  ceil_decimal(8 * LEMON_PER_PIECE['b'])
}
# 加工前在庫(原料檸檬)
initial_inventory_pre = {
    'a': ceil_decimal(2 * LEMON_PER_PIECE['a']),
    'b': ceil_decimal(3 * LEMON_PER_PIECE['b'])
    }

print(f"加工後在庫{initial_inventory_post}")
print(f"加工前在庫{initial_inventory_pre}")

num_scenarios_per_period = 20 # 生成するシナリオの数

# --- 環境負荷係数（ポイント/kg =  kgCO2e/kg に対応） ---
CO2_FACTOR_ELEC = 0.511 # 電気: 中国電力 (kg-CO2/kWh)
CO2_FACTOR_GAS = 3.00 # ガス: LPガス (kg-CO2/kg)

# 廃棄負荷 レモン1kgを製品に換算した場合の廃棄負荷 * (物理重量 / レモン含有量)
waste_emission_factors = {
    'middle': {
        'f': 1.76,
        'a': ceil_decimal(1.76 * (WEIGHT_PIECE['a'] / LEMON_PER_PIECE['a'])),
        'b': ceil_decimal(1.76 * (WEIGHT_PIECE['b'] / LEMON_PER_PIECE['b']))
    }
}
# 加工負荷
# タルト('a'): 5.9kW 0.5時間 (30個分)
emissions_energy_a = 5.9 * 0.5 * CO2_FACTOR_ELEC
e_p_a = ceil_decimal(emissions_energy_a / (30 * LEMON_PER_PIECE['a']))

# ジャム('b'): 0.915kg/h 1時間 (25個分)
emissions_energy_b = 0.915 * 1 * CO2_FACTOR_GAS
e_p_b = ceil_decimal(emissions_energy_b / (25 * LEMON_PER_PIECE['b']))

process_emission_factors = {'middle': {'a': e_p_a, 'b': e_p_b }}

print(f"廃棄負荷: {waste_emission_factors}")
print(f"加工負荷: {process_emission_factors}")


保管コスト: {'f': 2.79, 'a': 81.48, 'b': 2.0}
加工コスト: {'f': 0, 'a': 1301.3, 'b': 1216.98}
機会損失コスト: {'f': 500, 'a': 76000.0, 'b': 11000.0}
廃棄コスト: {'f': 22.8, 'a': 159.6, 'b': 45.6}
劣化率: {'f': 0.5, 'a': 0.3, 'b': 0.09}
需要パラメータ{'mean': {'f': 5.0, 'a': 6.0, 'b': 7.0}, 'std': {'f': 2.0, 'a': 2.0, 'b': 2.0}}
加工後在庫{'f': 6.0, 'a': 0.25, 'b': 1.6}
加工前在庫{'a': 0.1, 'b': 0.61}
廃棄負荷: {'middle': {'f': 1.76, 'a': 12.32, 'b': 3.52}}
加工負荷: {'middle': {'a': 1.01, 'b': 0.55}}


## パレート

In [14]:
from google.colab import auth
import gspread
from google.auth import default

def export_to_google_sheets(df, spreadsheet_title="Simulation_Results"):
    """
    Pandas DataFrameをGoogleスプレッドシートに出力する。
    """
    # 1. Googleアカウントの認証
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # 2. スプレッドシートの作成（または既存のものを開く）
    try:
        sh = gc.open(spreadsheet_title)
        print(f"既存のスプレッドシート '{spreadsheet_title}' を開きました。")
    except gspread.exceptions.SpreadsheetNotFound:
        # 新規作成
        sh = gc.create(spreadsheet_title)
        print(f"新規スプレッドシート '{spreadsheet_title}' を作成しました。")

    worksheet = sh.get_worksheet(0) # 1番目のシートを選択

    # 3. DataFrameをリスト形式に変換 (ヘッダー含む)
    # NaNや無限大などはJSON変換でエラーになるため、空文字や数値に置換
    processed_df = df.replace([np.inf, -np.inf], np.nan).fillna("")
    data_to_upload = [processed_df.columns.values.tolist()] + processed_df.values.tolist()

    # 4. シートの全内容をクリアして書き込み
    worksheet.clear()
    worksheet.update('A1', data_to_upload)

    print(f"データの書き込みが完了しました。")
    print(f"URL: https://docs.google.com/spreadsheets/d/{sh.id}")

In [15]:
# =================================================================
# モジュール3: 多目的最適化モデルを解く関数
# =================================================================
import shutil # ソルバーのパス確認用

# --- モジュール3-1 ---
def solve_multi_objective_model(initial_inv_post, initial_inv_pre, supply, scenarios_list, params, caps, env_weight, normalization_limits=None, sales_prices=None):
    model = pyo.ConcreteModel("Multi_Objective_Rolling_Model") # 最適化問題の箱を作る

    # --- 集合の定義( pyo.Set ) --- パラメータの値を入れている
    T_horizon = len(scenarios_list) # 何期分か
    model.T = pyo.Set(initialize=range(T_horizon)) # 期間の集合 t=0, 1, ..., T-1 initialize=初期化
    model.I = pyo.Set(initialize=params['h'].keys())
    model.I_proc = pyo.Set(initialize=[i for i in params['h'].keys() if i != 'f'])

    scenario_keys = scenarios_list[0].keys() # 1期ごとの全シナリオを入れる
    model.S = pyo.Set(initialize=scenario_keys)

    # --- 決定変数( pyo.Var ) --- GLPKに決めさせる変数
    # NonNegativeReals= 非負制約
    model.X = pyo.Var(model.T, model.I, domain=pyo.NonNegativeReals) # 原料配分量
    model.P_actual = pyo.Var(model.T, model.I_proc, domain=pyo.NonNegativeReals) # 加工投入量
    model.I_end_post = pyo.Var(model.T, model.S, model.I, domain=pyo.NonNegativeReals) # 期末在庫(加工後)
    model.I_end_pre = pyo.Var(model.T, model.S, model.I_proc, domain=pyo.NonNegativeReals) # 期末在庫(加工前)
    model.S_sold = pyo.Var(model.T, model.S, model.I, domain=pyo.NonNegativeReals) # 販売量
    model.W_waste_post = pyo.Var(model.T, model.S, model.I, domain=pyo.NonNegativeReals) # 廃棄量(加工後)
    model.W_waste_pre = pyo.Var(model.T, model.S, model.I_proc, domain=pyo.NonNegativeReals) # 廃棄量(加工前)
    model.W_waste_proc = pyo.Var(model.T, model.I_proc, domain=pyo.NonNegativeReals) # 加工廃棄
    model.UnmetDemand = pyo.Var(model.T, model.S, model.I, domain=pyo.NonNegativeReals) # 未充足需要
    model.waste_emission_factors = waste_emission_factors # 廃棄環境負荷
    model.process_emission_factors = process_emission_factors  # 加工環境負荷

    # --- 目的関数要素の定義 ---
    # 経済コスト
    def calc_economic_cost(m): # m = modelを受け取る 以降は「m.」で呼び出し
        total_cost = 0

        for t in m.T: # 最適化6カ月分
            scenarios = scenarios_list[t]

            # 保管コスト (for i in m.I) = 製品集合から１種ずつ取り出す
            term_h_post = sum(scenarios[s]['prob'] * sum(params['h'][i] * m.I_end_post[t, s, i] for i in m.I) for s in m.S)
            term_h_pre = sum(scenarios[s]['prob'] * sum(params['h']['f'] * m.I_end_pre[t, s, i] for i in m.I_proc) for s in m.S)

            # 加工コスト (生産活動そのもののコスト)
            term_p = sum(params['p'][i] * m.P_actual[t, i] for i in m.I_proc)

            # 廃棄コスト (劣化廃棄 + 加工廃棄)
            # 劣化廃棄 (確率的)
            term_w_deg = sum(scenarios[s]['prob'] * (
                sum(params['phi'][i] * m.W_waste_post[t, s, i] for i in m.I) +
                sum(params['phi']['f'] * m.W_waste_pre[t, s, i] for i in m.I_proc)
            ) for s in m.S)

            # 加工廃棄
            term_w_proc = sum(params['phi']['f'] * m.W_waste_proc[t, i] for i in m.I_proc)

            # 機会損失
            term_opp = sum(scenarios[s]['prob'] * sum(params['o'][i] * m.UnmetDemand[t, s, i] for i in m.I) for s in m.S)

            total_cost += (term_h_post + term_h_pre + term_p + term_w_deg + term_w_proc + term_opp)

        return total_cost

    # 環境負荷
    def calc_env_score(m):
        total_score = 0

        for t in m.T:
            scenarios = scenarios_list[t]

            # 廃棄負荷 (Middle)
            # 劣化廃棄
            term_env_w = sum(scenarios[s]['prob'] * (
                # 製品を捨てた時
                sum(m.waste_emission_factors['middle'][p] * m.W_waste_post[t, s, p] for p in m.I) +
                # 加工前在庫を捨てた時
                sum(m.waste_emission_factors['middle']['f'] * m.W_waste_pre[t, s, i] for i in m.I_proc)
            ) for s in m.S)

            # 加工廃棄
            term_env_w_proc = sum(m.waste_emission_factors['middle']['f'] * m.W_waste_proc[t, p] for p in m.I_proc)

            # 加工負荷 (エネルギー使用など)
            term_env_p = sum(m.process_emission_factors['middle'][p] * m.P_actual[t, p] for p in m.I_proc)

            total_score += (term_env_w + term_env_w_proc + term_env_p)
        return total_score

    # Expression = 数式
    model.economic_cost = pyo.Expression(rule=calc_economic_cost) # 経済コストの数式を登録する
    model.environmental_score = pyo.Expression(rule=calc_env_score) # 環境負荷の数式を登録する

    # 目的関数の正規化
    def objective_rule(m):
        cost_term = m.economic_cost
        env_term = m.environmental_score
        if normalization_limits:
            c_min, c_max, e_min, e_max = normalization_limits
            denom_c = c_max - c_min if (c_max - c_min) > 1e-5 else 1.0 # 差が小さい場合は分母を1.0にする
            denom_e = e_max - e_min if (e_max - e_min) > 1e-5 else 1.0
            norm_cost = (cost_term - c_min) / denom_c
            norm_env  = (env_term - e_min) / denom_e
            return (1.0 - env_weight) * norm_cost + env_weight * norm_env
        else:
            return (1.0 - env_weight) * cost_term + env_weight * env_term

    # 目的関数の数式を登録する
    model.total_objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize) # Objective = 目的関数のリスト

    # --- 制約条件 ---
    model.constraints = pyo.ConstraintList() # 制約のリストを表す制約コンポーネント

    for t in model.T:
        scenarios = scenarios_list[t]

        # 1. 総供給量制約（供給はすべて割り振る）
        # .add = 制約リストに追加する
        model.constraints.add(sum(model.X[t, i] for i in model.I) == supply)

        # 2. 加工能力と原料利用の制約
        for i in model.I_proc:
            # 加工ロス計算: 投入量 × (1 - 効率)
            model.constraints.add(model.W_waste_proc[t, i] == model.P_actual[t, i] * (1 - params['e'][i]))
            # 加工能力上限
            model.constraints.add(model.P_actual[t, i] <= caps['limit_units'][i])

            if t == 0:
                # 0期は「今あるレモン在庫」から劣化分を引いたもの ＋ 今期配分されたもの
                prev_pre_inv = initial_inv_pre.get(i, 0)
            else:
                # 来期以降は「前の期にこれくらい残るだろう」という期待値からスタート
                prev_pre_inv = sum(scenarios_list[t-1][s]['prob'] * model.I_end_pre[t-1, s, i] for s in model.S)

            # # 加工投入量は利用可能量以下（加工が強制ではない）
            # model.constraints.add(model.P_actual[t, i] <= prev_pre_inv * (1 - params['gamma']['f']) + model.X[t, i])

            # # シミュレーション|| 加工強制制約の追加
            # # 配分されたら全量投入する (在庫 pre は常に 0 になるように誘導)
            available_raw = prev_pre_inv * (1 - params['gamma']['f']) + model.X[t, i]
            model.constraints.add(model.P_actual[t, i] == available_raw)

        # 3. 各シナリオにおけるバランス制約
        for s in model.S:
            # --- 生鮮品 (f) ---
            if t == 0:
                start_inv_f = initial_inv_post['f']
            else:
                start_inv_f = sum(scenarios_list[t-1][ss]['prob'] * model.I_end_post[t-1, ss, 'f'] for ss in model.S)

            # 劣化廃棄量の確定
            model.constraints.add(model.W_waste_post[t, s, 'f'] == start_inv_f * params['gamma']['f'])

            # 在庫推移: (前期残り - 劣化) + 今期配分 == 販売量 + 今期末残り
            model.constraints.add(
                (start_inv_f - model.W_waste_post[t, s, 'f']) + model.X[t, 'f']
                == model.S_sold[t, s, 'f'] + model.I_end_post[t, s, 'f']
            )

            # --- 加工品 (a, b) ---
            for i in model.I_proc:
                if t == 0:
                    start_inv_post = initial_inv_post.get(i, 0)
                    start_inv_pre  = initial_inv_pre.get(i, 0)
                else:
                    start_inv_post = sum(scenarios_list[t-1][ss]['prob'] * model.I_end_post[t-1, ss, i] for ss in model.S)
                    start_inv_pre  = sum(scenarios_list[t-1][ss]['prob'] * model.I_end_pre[t-1, ss, i] for ss in model.S)

                # 加工済み製品(post)の動き
                # 劣化廃棄
                model.constraints.add(model.W_waste_post[t, s, i] == start_inv_post * params['gamma'][i])
                # 在庫推移: (前期製品残り - 劣化) + 今期加工完了分 == 販売量 + 今期末製品残り
                model.constraints.add(
                    (start_inv_post - model.W_waste_post[t, s, i]) + (model.P_actual[t, i] * params['e'][i])
                    == model.S_sold[t, s, i] + model.I_end_post[t, s, i]
                )

                # 加工待ち原料(pre)の動き
                # 原料劣化
                model.constraints.add(model.W_waste_pre[t, s, i] == start_inv_pre * params['gamma']['f'])
                # 在庫推移: (前期原料残り - 劣化) + 今期原料配分 - 今期加工投入 == 今期末原料残り
                model.constraints.add(
                    (start_inv_pre - model.W_waste_pre[t, s, i]) + model.X[t, i] - model.P_actual[t, i]
                    == model.I_end_pre[t, s, i]
                )

            # --- 需要・廃棄・上限 ---
            for i in model.I:
                # 需要 ＝ 販売量 ＋ 売り逃し
                model.constraints.add(model.S_sold[t, s, i] + model.UnmetDemand[t, s, i] == scenarios[s]['d'][i])
                # 売れる量は需要が上限
                model.constraints.add(model.S_sold[t, s, i] <= scenarios[s]['d'][i])
                # 保管庫の容量上限
                model.constraints.add(model.I_end_post[t, s, i] <= caps['U'][i])

    # --- ソルバーの実行 ---
    # グローバル変数のsolverではなく、関数内で確実にglpkを見つけて実行する
    glpsol_path = shutil.which("glpsol")
    if glpsol_path:
        opt = pyo.SolverFactory('glpk', executable=glpsol_path) # Pyomoが書いた数式モデルを読み、最適化を実行する
    else:
        # パスが見つからない場合はデフォルト動作
        opt = pyo.SolverFactory('glpk')

    try:
        results = opt.solve(model, tee=False) # tee=Trueでソルバーログが出る
    except Exception as e:
        print(f"Solver Error: {e}")
        print("ランタイムがリセットされた可能性 !apt-get install glpk-utils を再実行してください。")
        return None, None

    # results = メタ情報、.status = 動いたか(ok=OK)、.termination_conditio = 解が見つかったか(optimal=最適解見つかった)
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        decisions = {
            # 製品iごとに、今期(t=0)の原料配分量を数値として取り出す
            'X': {i: pyo.value(model.X[0, i]) for i in model.I},
            # 加工投入量
            'P_actual': {i: pyo.value(model.P_actual[0, i]) for i in model.I_proc}
        }
        # decisions= 次期の在庫更新用、model= Pyomoモデル丸ごと（評価用）
        return decisions, model
    else:
        return None, None


# --- モジュール3-2 : 正規化の基準値を探す ---
def calculate_normalization_bounds(initial_inv_post, initial_inv_pre, supply, scenarios_list, params, caps, weight_grid=None):
    print(f"正規化のための基準値を計算中 (Horizon: {len(scenarios_list)} months)...")

    if weight_grid is None:
        # 正規化用の粗い探索グリッド（標準的）
        # 重みの候補リスト、np.linspace(開始,終了,間を何等分するか)
        weight_grid = np.linspace(0.0, 1.0, 11)

    print(f"正規化のための基準値を計算中 "
          f"(Horizon: {len(scenarios_list)} months, "
          f"Grid size: {len(weight_grid)})...")

    cost_values = []
    env_values = []

    # 正規化なしで最適化を回す、ローリングホライズンなし
    # 6カ月分を一括でλ(0～1)変化させる
    for w in weight_grid:
        _, model = solve_multi_objective_model( # モジュール3-1（最適化）
                                                # _, = １個目の返り値(decisions)を使用しない
            initial_inv_post, # 初期在庫
            initial_inv_pre, # 加工前初期在庫
            supply, # 供給量
            scenarios_list, # 確率シナリオ
            params, # コストパラメータ
            caps, # 制約
            env_weight=w, # 重み
            normalization_limits=None  # ← 正規化しない
        )

        if model is None:
            raise RuntimeError(f"ソルバーエラー (env_weight={w})")

        cost = pyo.value(model.economic_cost)
        env  = pyo.value(model.environmental_score)

        cost_values.append(cost)
        env_values.append(env)

    # 現実的な限界値を記録
    c_min = min(cost_values)
    c_max = max(cost_values)
    e_min = min(env_values)
    e_max = max(env_values)

    print(f"  Cost Range (Total for Horizon): {c_min:,.0f} ~ {c_max:,.0f}")
    print(f"  Env  Range (Total for Horizon): {e_min:,.3f} ~ {e_max:,.3f}")

    return c_min, c_max, e_min, e_max

# =================================================================
# モジュール4: シミュレーション (実績値詳細計算・ローリングホライズン)
# =================================================================
def run_two_stage_simulation(simulation_periods, env_weight, initial_inventory_post, initial_inventory_pre, master_scenarios, normalization_limits=None, verbose=True):
    if verbose:
        print("\n" + "="*80)
        print(f"シミュレーション実行中 (ローリングホライズン T={HORIZON_LENGTH}, λ={env_weight:.2f})")
        print("="*80)

    HORIZON_LENGTH = 6 # 計画期間 (当期+未来5ヶ月)
    # 初期在庫の設定
    inv_post = initial_inventory_post.copy()
    inv_pre = initial_inventory_pre.copy()
    full_history = []

    for t in range(1, simulation_periods + 1):
        # 1. ローリングホライズン用のシナリオ切り出し (当期 t から Tヶ月分)
        current_scenarios_list = []
        for h in range(HORIZON_LENGTH):
            target_idx = (t - 1) + h
            # シナリオが足りない場合は最終期間のものをコピーして埋める（エラー対策）
            if target_idx < len(master_scenarios):
                current_scenarios_list.append(master_scenarios[target_idx])
            else:
                current_scenarios_list.append(master_scenarios[-1])

        # 2. 最適化実行 (6ヶ月先まで考慮して決定)
        decisions, solved_model = solve_multi_objective_model( # モジュール3-1（最適化）
            inv_post, inv_pre, C_supply, current_scenarios_list,
            params, capacities, env_weight, normalization_limits
        )

        if not decisions:
            print(f"❌ {t}期で最適解なし。中断。")
            break
        # 3. 実績値の計算
        scenarios = current_scenarios_list[0] # 当期のシナリオ

        # 決定変数
        # i:ループ、value:数値変換、solved_model:計算済みモデル、products:全製品タイプ
        val_X = {i: pyo.value(solved_model.X[0, i]) for i in products}
        val_P = {i: pyo.value(solved_model.P_actual[0, i]) for i in processed_products}

        # 在庫量の期待値
        val_Inv_post = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_post[0, s, i]) for s in scenarios) for i in products}
        val_Inv_pre  = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_pre[0, s, i]) for s in scenarios) for i in processed_products}

        # 劣化廃棄
        val_W_post = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_post[0, s, i]) for s in scenarios) for i in products}
        val_W_pre  = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_pre[0, s, i]) for s in scenarios) for i in processed_products}

        # 加工廃棄
        val_W_proc = {i: pyo.value(solved_model.W_waste_proc[0, i]) for i in processed_products}
        val_Unmet = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.UnmetDemand[0, s, i]) for s in scenarios) for i in products}

        # コスト計算
        cost_holding = sum(params['h'][i] * val_Inv_post[i] for i in products) + sum(params['h']['f'] * val_Inv_pre[i] for i in processed_products)
        cost_processing = sum(params['p'][i] * val_P[i] for i in processed_products)

        # 廃棄コスト (劣化 + 加工)
        cost_waste_deg = sum(params['phi'][i] * val_W_post[i] for i in products) + sum(params['phi']['f'] * val_W_pre[i] for i in processed_products)
        cost_waste_proc = sum(params['phi'][i] * val_W_proc[i] for i in processed_products)
        cost_waste = cost_waste_deg + cost_waste_proc

        cost_opportunity = sum(params['o'][i] * val_Unmet[i] for i in products)
        cost_total = cost_holding + cost_processing + cost_waste + cost_opportunity

        # 環境負荷計算
        # 廃棄由来 (劣化 + 加工)
        env_waste_deg = sum(waste_emission_factors['middle'][i] * val_W_post[i] for i in products) + sum(waste_emission_factors['middle']['f'] * val_W_pre[i] for i in processed_products)
        env_waste_proc = sum(waste_emission_factors['middle'][i] * val_W_proc[i] for i in processed_products)
        env_waste = env_waste_deg + env_waste_proc

        env_process = sum(process_emission_factors['middle'][i] * val_P[i] for i in processed_products)
        env_total = env_waste + env_process

        # 需要の期待値
        val_Exp_Demand = {i: sum(scenarios[s]['prob'] * scenarios[s]['d'][i] for s in scenarios) for i in products}
        # 未充足需要の期待値
        val_Unmet = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.UnmetDemand[0, s, i]) for s in scenarios) for i in products}

        # データ記録
        period_data = {
            'Period': t,
            'Exp_Demand_f': val_Exp_Demand['f'], 'Exp_Demand_a': val_Exp_Demand['a'], 'Exp_Demand_b': val_Exp_Demand['b'],
            'Total Cost': cost_total,
            'Holding Cost': cost_holding, 'Processing Cost': cost_processing,
            'Waste Cost': cost_waste, 'Opportunity Cost': cost_opportunity,
            'Total Env Score': env_total, 'Waste Env': env_waste, 'Process Env': env_process,
            'Inv_post_f': val_Inv_post['f'], 'Inv_post_a': val_Inv_post['a'], 'Inv_post_b': val_Inv_post['b'],
            'Inv_pre_a': val_Inv_pre['a'], 'Inv_pre_b': val_Inv_pre['b'],
            'X_f': val_X['f'], 'X_a': val_X['a'], 'X_b': val_X['b'],
            'P_a': val_P['a'], 'P_b': val_P['b'],
            'W_post_f': val_W_post['f'], 'W_post_a': val_W_post['a'], 'W_post_b': val_W_post['b'],
            'W_pre_a': val_W_pre['a'], 'W_pre_b': val_W_pre['b'],
            'W_proc_a': val_W_proc['a'], 'W_proc_b': val_W_proc['b'],
            'Unmet_f': val_Unmet['f'], 'Unmet_a': val_Unmet['a'], 'Unmet_b': val_Unmet['b'],
        }
        full_history.append(period_data) # データを配列に追加する

        # 4. 次期への在庫更新
        inv_post = {p: period_data[f'Inv_post_{p}'] for p in products}
        inv_pre = {p: period_data[f'Inv_pre_{p}'] for p in processed_products}

    # pd:Pandas、DataFrame:行列の表に整形する
    return pd.DataFrame(full_history)

# =================================================================
# モジュール5: パレートフロンティア分析
# =================================================================
def run_pareto_analysis(env_weight_list, simulation_periods, master_scenarios, normalization_limits=None, title="パレートフロンティア分析"):
    print("="*80 + f"\n{title}を開始...\n" + "="*80)
    results_list = []

    for weight in env_weight_list:
        # 各λでシミュレーションを実行
        # モジュール4（ローリングホライズン）を試行、λを変化させつつ12ヶ月分
        df_history = run_two_stage_simulation(simulation_periods, env_weight=weight,
                                                  initial_inventory_post=initial_inventory_post,
                                                  initial_inventory_pre=initial_inventory_pre,
                                                  master_scenarios=master_scenarios,
                                                  normalization_limits=normalization_limits,
                                                  verbose=False)
        if not df_history.empty:
            # シミュレーション結果(df_history)= 実績コストのため、合計するだけでよい
            total_cost = df_history['Total Cost'].sum()
            total_env = df_history['Total Env Score'].sum()

            results_list.append({
                'weight': weight,
                'total_cost': total_cost,
                'total_env_score': total_env,
                'total_objective': (1 - weight) * total_cost + weight * total_env
            })

    return pd.DataFrame(results_list)

# =================================================================
# モジュール6: 1期間の詳細分析と結果表示
# =================================================================
def analyze_and_print_period_details(model, scenarios, params):
    t = 0 # 最適化モデル内の t=0 (当期) の情報を表示

    # コスト計算
    holding_cost_post = sum(scenarios[s]['prob'] * sum(params['h'][i] * pyo.value(model.I_end_post[t, s, i]) for i in model.I) for s in model.S)
    holding_cost_pre = sum(scenarios[s]['prob'] * sum(params['h']['f'] * pyo.value(model.I_end_pre[t, s, i]) for i in model.I_proc) for s in model.S)
    processing_cost = sum(params['p'][i] * pyo.value(model.P_actual[t, i]) for i in model.I_proc)

    # 廃棄コスト (劣化)
    waste_cost_deg = sum(scenarios[s]['prob'] * (
        sum(params['phi'][i] * pyo.value(model.W_waste_post[t, s, i]) for i in model.I) +
        sum(params['phi']['f'] * pyo.value(model.W_waste_pre[t, s, i]) for i in model.I_proc)
    ) for s in model.S)

    # 廃棄コスト (加工)
    waste_cost_proc = sum(params['phi'][i] * pyo.value(model.W_waste_proc[t, i]) for i in model.I_proc)

    opportunity_loss_cost = sum(scenarios[s]['prob'] * sum(params['o'][i] * pyo.value(model.UnmetDemand[t, s, i]) for i in model.I) for s in model.S)
    total_cost_period = holding_cost_post + holding_cost_pre + processing_cost + waste_cost_deg + waste_cost_proc + opportunity_loss_cost

    # ロス量計算
    spoilage_f = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[t, s, 'f']) for s in model.S)
    spoilage_post_a = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[t, s, 'a']) for s in model.S)
    spoilage_pre_a = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_pre[t, s, 'a']) for s in model.S)
    processing_loss_a = pyo.value(model.W_waste_proc[t, 'a']) # 直接変数から取得

    spoilage_post_b = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[t, s, 'b']) for s in model.S)
    spoilage_pre_b = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_pre[t, s, 'b']) for s in model.S)
    processing_loss_b = pyo.value(model.W_waste_proc[t, 'b']) # 直接変数から取得

    total_food_loss = spoilage_f + spoilage_post_a + spoilage_pre_a + spoilage_post_b + spoilage_pre_b + processing_loss_a + processing_loss_b

    # 環境負荷計算 (Middle)
    level = 'middle'
    waste_impact_deg = sum(scenarios[s]['prob'] * (
        sum(waste_emission_factors[level][i] * pyo.value(model.W_waste_post[t, s, i]) for i in model.I) +
        sum(waste_emission_factors[level]['f'] * pyo.value(model.W_waste_pre[t, s, i]) for i in model.I_proc)
    ) for s in model.S)
    waste_impact_proc = sum(waste_emission_factors[level][i] * pyo.value(model.W_waste_proc[t, i]) for i in model.I_proc)

    process_impact = sum(process_emission_factors[level][i] * pyo.value(model.P_actual[t, i]) for i in model.I_proc)
    total_impact = waste_impact_deg + waste_impact_proc + process_impact

    # 表示
    print(f"期待総コスト (当期分): {total_cost_period:,.3f} 円")
    print("\n--- 総コストの内訳 (当期分) ---")
    print(f"  期待保管コスト        : {(holding_cost_post + holding_cost_pre):,.3f}")
    print(f"  加工コスト            : {processing_cost:,.3f}")
    print(f"  期待廃棄コスト        : {(waste_cost_deg + waste_cost_proc):,.3f} (内 加工廃棄: {waste_cost_proc:,.3f})")
    print(f"  期待機会損失コスト    : {opportunity_loss_cost:,.3f}")
    print(f"総食品ロス量            : {total_food_loss:.3f} kg (内 加工ロス: {(processing_loss_a + processing_loss_b):.3f} kg)")
    print(f"Middle 環境負荷         : {total_impact:.3f} ポイント")

# =================================================================
# モジュール7: ダッシュボード
# =================================================================
def create_dashboard(df, title_suffix=""):
    if df.empty: return

    UNIQUE_COLORS = ['#4E79A7', '#FF9DA7', '#59A14F']

    COLOR_F = '#228B22' # フォレストグリーン (生鮮f: 中間の濃さ)
    COLOR_A = '#E13939' # ブライトレッド (タルトa: 明るめ)
    COLOR_B = '#1F4E79' # コバルトブルー (シロップb: 最も濃い)

    # 加工前(原料)用の薄い色
    COLOR_A_PRE = '#F5B7B1' # 薄い赤
    COLOR_B_PRE = '#A9CCE3' # 薄い青

    # 全体で統一して使うためのカラーマッピング
    color_map = {
        'f': COLOR_F,
        'a': COLOR_A,
        'b': COLOR_B,
        'pre_a': COLOR_A_PRE,
        'pre_b': COLOR_B_PRE
    }

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("在庫レベル推移 (kg)", "加工品生産量 (kg)", "廃棄量の内訳 (kg)", "原料の用途別配分推移 (kg)"),
        specs=[[{}, {}], [{}, {}]]
    )

    inv_post_cols = ['Inv_post_f', 'Inv_post_a', 'Inv_post_b']
    inv_pre_cols = ['Inv_pre_a', 'Inv_pre_b']
    colors = px.colors.qualitative.Plotly

    # 1. 在庫グラフ (左上)
    for i, col in enumerate(inv_post_cols):
        fig.add_trace(go.Bar(x=df['Period'], y=df[col] , name=f'加工後-{col[-1]}', legendgroup='1_inv', marker_color=color_map[col[-1]]), row=1, col=1)
    for i, col in enumerate(inv_pre_cols):
         fig.add_trace(go.Bar(x=df['Period'], y=df[col], name=f'加工前-{col[-1]}', legendgroup='1_inv', marker_color=color_map[col[-1]], opacity=0.7), row=1, col=1)

    # 2. 生産量グラフ (右上)
    fig.add_trace(go.Bar(x=df['Period'], y=df['P_a'], name='タルト(a)生産', legendgroup='2_prod', marker_color=colors[1]), row=1, col=2)
    fig.add_trace(go.Bar(x=df['Period'], y=df['P_b'], name='シロップ(b)生産', legendgroup='2_prod', marker_color=colors[2]), row=1, col=2)

    # 3. 廃棄量グラフ (左下)
    waste_cols = ['W_post_f', 'W_post_a', 'W_post_b', 'W_pre_a', 'W_pre_b', 'W_proc_a', 'W_proc_b']
    waste_names = { 'W_post_f': '生鮮廃棄(f)', 'W_post_a': '製品廃棄(a)', 'W_post_b': '製品廃棄(b)', 'W_pre_a': '原料劣化(a)', 'W_pre_b': '原料劣化(b)', 'W_proc_a': '加工ロス(a)', 'W_proc_b': '加工ロス(b)'}
    for i, col in enumerate(waste_cols):
        # 該当カラムが存在する場合のみ描画
        if col in df.columns:
            fig.add_trace(go.Bar(
                x=df['Period'], y=df[col],
                name=waste_names.get(col, col),
                legendgroup='3_waste',
                marker_color=px.colors.qualitative.Pastel[i%10]
            ), row=2, col=1)

    # 4. 原料配分推移グラフ (右下) - 積み上げ棒グラフ
    alloc_cols = ['X_f', 'X_a', 'X_b']
    alloc_names = {'X_f': '配分-生鮮(f)', 'X_a': '配分-タルト用(a)', 'X_b': '配分-シロップ用(b)'}

    for i, col in enumerate(alloc_cols):
        fig.add_trace(go.Bar(
            x=df['Period'], y=df[col],
            name=alloc_names[col],
            legendgroup='4_alloc',
            marker_color=colors[i] # 在庫グラフの色と合わせる
        ), row=2, col=2)

    fig.update_layout(
        height=800,
        title_text=f"詳細ダッシュボード {title_suffix}",
        legend_title_text="凡例",
        barmode='stack',
        bargap=0.15
    )

    # 軸ラベルの設定
    fig.update_xaxes(title_text="期間 (Month)", tickmode='linear')
    fig.update_yaxes(title_text="重量 (kg)", row=1, col=1)
    fig.update_yaxes(title_text="重量 (kg)", row=1, col=2)
    fig.update_yaxes(title_text="重量 (kg)", row=2, col=1)
    fig.update_yaxes(title_text="重量 (kg)", row=2, col=2)

    fig.show()

# =================================================================
# モジュール8: メイン実行ブロック トップレベル環境 一番最初に実行される
# =================================================================
if __name__ == '__main__':

    np.random.seed(42)

    SIMULATION_PERIODS = 12
    PLANNING_HORIZON = 6
    TOTAL_SCENARIO_PERIODS = SIMULATION_PERIODS + PLANNING_HORIZON # 合計18カ月分

    # 需要シナリオ生成
    print("="*80)
    print(f"シナリオ生成中 (期間: {SIMULATION_PERIODS}, Horizon: {PLANNING_HORIZON})...")
    master_scenarios = []
    for t in range(TOTAL_SCENARIO_PERIODS):
        scenarios_for_period_t = {
            f"s_{i+1}": {
                'prob': 1 / num_scenarios_per_period, # そのシナリオが起こる確率
                'd': {p: max(0, np.random.normal(demand_params['mean'][p], demand_params['std'][p])) for p in products} # 需要生成
            } for i in range(num_scenarios_per_period)
        }
        master_scenarios.append(scenarios_for_period_t)

    # 正規化基準値の計算
    try:
        norm_limits = calculate_normalization_bounds( # モジュール3-2（正規化の基準値を探す）
            initial_inventory_post, initial_inventory_pre, C_supply, master_scenarios[0:PLANNING_HORIZON], params, capacities
        )
    except Exception as e:
        print(f"Error during normalization: {e}")
        norm_limits = None

    if norm_limits:
        # --- パレートフロンティア分析 ---
        print("\n" + "="*80)
        print("パレートフロンティア分析を実行中 (λ: 0.0 -> 1.0)")
        print("="*80)

   # グラフ描画に必要な変数の定義
        grid_color_mono = 'darkgray'

        # np.linspace(開始,終了,間を何等分するか)
        env_weights_linear = np.linspace(0.0,1.0,11)
        # モジュール5（パレートフロンティア分析）
        df_results_linear = run_pareto_analysis(env_weights_linear, SIMULATION_PERIODS, master_scenarios, norm_limits, title="詳細スキャン")

        if not df_results_linear.empty:
            print("\n--- λ（環境重視度）と各指標の関係 ---")
            fig_lambda_linear_separate = make_subplots(
                rows=2, cols=1,shared_xaxes=True,vertical_spacing=0.08,
            )

            # 総コスト
            fig_lambda_linear_separate.add_trace(go.Scatter(
                x=df_results_linear['weight'], y=df_results_linear['total_cost'],
                name='総コスト', mode='lines+markers',
                line=dict( dash='solid', width=2),
                marker=dict(symbol='circle', size=6)
            ), row=1, col=1)

            # 総環境負荷
            fig_lambda_linear_separate.add_trace(go.Scatter(
                x=df_results_linear['weight'], y=df_results_linear['total_env_score'],
                name='総環境負荷', mode='lines+markers',
                line=dict(dash='solid', width=2),
                marker=dict(symbol='circle', size=6)
            ), row=2, col=1)

            fig_lambda_linear_separate.update_layout(
                    # title_text=f'λ=0～1における各指標の関係',
                    height=600, showlegend=False,
                    plot_bgcolor='white', paper_bgcolor='white',title_font=dict(size=20), font_color='black',
                    margin=dict(l=110, r=30, t=30, b=100), autosize=True
            )

            # Y軸 (1段目)
            fig_lambda_linear_separate.update_yaxes(
                  title_text="総コスト（C）", title_standoff=20, title_font=dict(size=16, color='black'), tickfont=dict(size=14, color='black'),
                  showgrid=True,gridcolor=grid_color_mono,  zeroline=False, linecolor='black',
                  row=1, col=1
            )
            # Y軸 (2段目)
            fig_lambda_linear_separate.update_yaxes(
                  title_text="総環境負荷スコア（E）", title_standoff=20, title_font=dict(size=16, color='black'),  tickfont=dict(size=14, color='black'),
                  showgrid=True,gridcolor=grid_color_mono,zeroline=False,linecolor='black',
                  row=2, col=1
            )

            # X軸 (共通)
            fig_lambda_linear_separate.update_xaxes(
                  title_text='λ (環境重視度)',title_font=dict(size=16, color='black'),tickfont=dict(size=14, color='black'),
                  showgrid=False,gridcolor=grid_color_mono,linecolor='black',zeroline=False,
                  row=2, col=1 # 最後のグラフにのみX軸タイトルを表示
            )
            fig_lambda_linear_separate.show()


            # --- パレートフロンティア ---
            print("\n--- グラフ２: パレートフロンティア（コストと環境負荷のトレードオフ） ---")
            fig_pareto_frontier = go.Figure()
            fig_pareto_frontier.add_trace(go.Scatter(
                x=df_results_linear['total_env_score'],y=df_results_linear['total_cost'],mode='lines+markers',
                text=df_results_linear['weight'].apply(lambda w: f'λ={w:.1f}'), # ホバーテキスト
                hovertemplate='<b>環境負荷</b>: %{x:.2f}<br><b>総コスト</b>: %{y:,.0f} 円<br><b>%{text}</b><extra></extra>',
                line=dict(color='green'),   # 線の色
                marker=dict(color='green')  # 点の色
            ))

            fig_pareto_frontier.update_layout(
                # title_text='パレートフロンティア（コストと環境負荷のトレードオフ）',
                xaxis_title='総環境負荷スコア (E)',
                yaxis_title='総コスト (C)',
                height=500,showlegend=False,title_font=dict(size=20),
                plot_bgcolor='white',     # グラフ背景を白に
                paper_bgcolor='white',    # グラフ全体の背景を白に
                font_color='black',       # フォントを黒に
                margin=dict(l=120, r=30, t=30, b=100), autosize=True,
                # X軸のスタイル
                xaxis=dict(showgrid=True,gridcolor=grid_color_mono,linecolor='black',zeroline=False),
                # Y軸のスタイル
                yaxis=dict(showgrid=True,gridcolor=grid_color_mono,linecolor='black',zeroline=False)
            )

            fig_pareto_frontier.show()

            print("\n" + "="*80)
            print("【比較表】")
            print("="*80)
            print(df_results_linear[['weight', 'total_cost', 'total_env_score']].round(2))

        target_lambdas = [0.1, 0.5, 0.9]
        for chosen_weight in target_lambdas:

          # --- 詳細分析 (λ=0.5) ---
          print("\n" + "="*80)
          print(f"詳細分析 (λ={chosen_weight})")
          print("="*80)

          # 1期間目モデル詳細
          # モジュール3-1（最適化）λ=0.5の12カ月のみを再度計算する
          _, first_period_model = solve_multi_objective_model(initial_inventory_post, initial_inventory_pre, C_supply, master_scenarios[0:PLANNING_HORIZON], params, capacities, chosen_weight, norm_limits)
          # モジュール6（1期間の詳細）
          if first_period_model: analyze_and_print_period_details(first_period_model, master_scenarios[0], params)

          # 12期間シミュレーション & 論文用テーブル
          # モジュール4（ローリングホライズン)を試行、選択したλで1回のみ
          detailed_df = run_two_stage_simulation(SIMULATION_PERIODS, chosen_weight, initial_inventory_post, initial_inventory_pre, master_scenarios, norm_limits, verbose=False)
          if not detailed_df.empty:
              print("\n" + "="*100)
              print(f"【詳細データ表】 (λ = {chosen_weight}, Environment Factor: Middle)")
              print("="*100)

              # 論文用フォーマット
              display_df = detailed_df[['Period', 'Total Cost', 'Holding Cost', 'Processing Cost', 'Waste Cost', 'Opportunity Cost', 'Total Env Score', 'Waste Env', 'Process Env']].copy()
              display_df.columns = ['期(t)', '総コスト', '保管費用', '加工費用', '廃棄費用', '機会損失', '環境負荷(総)', '環境(廃棄)', '環境(加工)']

              pd.set_option('display.max_columns', None)
              pd.set_option('display.width', 1000)
              pd.set_option('display.float_format', '{:,.2f}'.format)

              print(display_df.to_string(index=False))
              print("-" * 100)

              totals = display_df.sum(numeric_only=True)
              totals['期(t)'] = float('nan')
              print(f"合計{'':<4} {totals['総コスト']:,.2f}   {totals['保管費用']:,.2f}     {totals['加工費用']:,.2f}     {totals['廃棄費用']:,.2f}    {totals['機会損失']:,.2f}    {totals['環境負荷(総)']:,.2f}     {totals['環境(廃棄)']:,.2f}     {totals['環境(加工)']:,.2f}")
              print("="*100)

              print(f"Plotting Dashboard for λ = {chosen_weight}...")
              create_dashboard(detailed_df, title_suffix=f"(λ={chosen_weight})")

              if chosen_weight == 0.5:
                export_to_google_sheets(detailed_df, "卒論_シミュレーション検証結果_λ0.5")


シナリオ生成中 (期間: 12, Horizon: 6)...
正規化のための基準値を計算中 (Horizon: 6 months)...
正規化のための基準値を計算中 (Horizon: 6 months, Grid size: 11)...
  Cost Range (Total for Horizon): 145,090 ~ 2,188,224
  Env  Range (Total for Horizon): 127.090 ~ 219.068

パレートフロンティア分析を実行中 (λ: 0.0 -> 1.0)
詳細スキャンを開始...

--- λ（環境重視度）と各指標の関係 ---



--- グラフ２: パレートフロンティア（コストと環境負荷のトレードオフ） ---



【比較表】
    weight   total_cost  total_env_score
0     0.00   294,768.09           861.68
1     0.10   371,908.44           770.64
2     0.20   468,747.34           733.77
3     0.30   624,592.18           694.53
4     0.40   792,707.92           664.76
5     0.50 1,053,853.44           625.45
6     0.60 1,191,226.26           609.02
7     0.70 1,398,560.40           586.99
8     0.80 1,744,036.84           556.36
9     0.90 2,338,709.81           510.00
10    1.00 2,768,141.69           480.64

詳細分析 (λ=0.1)
期待総コスト (当期分): 34,850.783 円

--- 総コストの内訳 (当期分) ---
  期待保管コスト        : 202.302
  加工コスト            : 25,347.194
  期待廃棄コスト        : 648.926 (内 加工廃棄: 553.896)
  期待機会損失コスト    : 8,652.360
総食品ロス量            : 7.717 kg (内 加工ロス: 4.143 kg)
Middle 環境負荷         : 66.050 ポイント

【詳細データ表】 (λ = 0.1, Environment Factor: Middle)
 期(t)      総コスト   保管費用      加工費用   廃棄費用      機会損失  環境負荷(総)  環境(廃棄)  環境(加工)
    1 34,850.78 202.30 25,347.19 648.93  8,652.36    66.05   50.09   15.96
    2 30,268.65 226.57 25,


詳細分析 (λ=0.5)
期待総コスト (当期分): 83,870.828 円

--- 総コストの内訳 (当期分) ---
  期待保管コスト        : 87.159
  加工コスト            : 20,064.650
  期待廃棄コスト        : 491.071 (内 加工廃棄: 396.041)
  期待機会損失コスト    : 63,227.948
総食品ロス量            : 6.663 kg (内 加工ロス: 3.089 kg)
Middle 環境負荷         : 50.123 ポイント

【詳細データ表】 (λ = 0.5, Environment Factor: Middle)
 期(t)       総コスト   保管費用      加工費用   廃棄費用       機会損失  環境負荷(総)  環境(廃棄)  環境(加工)
    1  83,870.83  87.16 20,064.65 491.07  63,227.95    50.12   37.91   12.22
    2  90,211.36  81.01 20,843.38 554.93  68,732.04    55.66   42.84   12.82
    3  80,300.13  82.19 20,629.50 514.04  59,074.39    52.02   39.68   12.34
    4  85,479.74  79.86 16,890.65 499.16  68,010.08    49.21   38.53   10.68
    5  83,777.01  80.51 19,488.57 486.19  63,721.74    48.95   37.53   11.42
    6  72,661.18  71.25 18,716.41 516.50  53,357.02    51.30   39.87   11.43
    7  85,915.68  99.42 22,093.30 589.28  63,133.69    59.06   45.49   13.57
    8 101,391.99 106.27 20,198.75 564.44  80,522.53    56.1

既存のスプレッドシート '卒論_シミュレーション検証結果_λ0.5' を開きました。


/tmp/ipython-input-609729256.py:32: DeprecationWarning:

The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)



データの書き込みが完了しました。
URL: https://docs.google.com/spreadsheets/d/1qhqG7-2wZxUKZ6WLeNRMSbosmUKIW3WpSfk_-PYZHaI

詳細分析 (λ=0.9)
期待総コスト (当期分): 245,595.737 円

--- 総コストの内訳 (当期分) ---
  期待保管コスト        : 40.311
  加工コスト            : 14,408.553
  期待廃棄コスト        : 290.252 (内 加工廃棄: 195.221)
  期待機会損失コスト    : 230,856.622
総食品ロス量            : 5.392 kg (内 加工ロス: 1.818 kg)
Middle 環境負荷         : 30.303 ポイント

【詳細データ表】 (λ = 0.9, Environment Factor: Middle)
 期(t)       総コスト  保管費用      加工費用   廃棄費用       機会損失  環境負荷(総)  環境(廃棄)  環境(加工)
    1 245,595.74 40.31 14,408.55 290.25 230,856.62    30.30   22.41    7.90
    2 186,422.56 53.01 18,163.62 481.19 167,724.75    47.99   37.14   10.84
    3 177,825.79 44.47 17,272.04 427.56 160,081.72    42.87   33.00    9.86
    4 180,098.48 46.74 14,698.34 428.36 164,925.04    41.92   33.07    8.85
    5 147,337.97 55.39 17,679.75 443.55 129,159.28    44.30   34.24   10.06
    6 141,578.84 44.84 16,625.56 460.84 124,447.59    45.41   35.57    9.84
    7 271,628.56 47.01 15,978.17 39